In [1]:
load('../Imports/Utility.sage', '../Imports/Point.sage', '../Imports/Line.sage', '../Imports/TritangentPlane.sage', '../Imports/Group.sage', '../Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [2]:
xx = QQ['xx'].0
Q.<i_rad3> = NumberField(xx^2+3)

In [3]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(Q)
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f) * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [4]:
# Define cubic SE18, find and classify the lines and compute the 45 tritangent planes

In [5]:
line = Line([y, z])
general_cubic = Cubic(cubic_new, line, sing_cubics)
%time SE18 = general_cubic.subs({b:-(c*c+e*f)/c, d:(3*e*f - c^2 + c*f + c*e)/(2*c)}).subs({f:c*(3*c-e)/(c+e)}).subs({e : i_rad3*c})

CPU times: user 4.8 s, sys: 39.2 ms, total: 4.84 s
Wall time: 4.85 s


In [6]:
# Find simmetries

In [7]:
%time adm_SE18 = SE18.find_admissible_projectivities()
len(adm_SE18)

CPU times: user 1.76 s, sys: 88.1 ms, total: 1.85 s
Wall time: 17 s


648

In [8]:
%time simm_SE18 = SE18.find_simmetries(adm_SE18)
len(simm_SE18)

CPU times: user 1.6 s, sys: 204 ms, total: 1.8 s
Wall time: 2.52 s


648

In [9]:
# Study how the simmetries permute the Eckardt points

In [9]:
Eck_perms = [Permutation(SE18.apply_proj_to_eck(proj[0])).to_permutation_group_element() for proj in simm_SE18]
Eck_perms_group = PermutationGroup(Eck_perms)
Eck_perms_group.order(), Eck_perms_group.structure_description()

(648, '(((C3 x C3 x C3) : (C2 x C2)) : C3) : C2')

In [10]:
# Sage's description is not too helpful, but we actually know that the group 
# is isomorphic to (C3 X C3 X C3) : S4
# What is this S4? It is the group of permutations of the four planes containing a C9 configuration
# i.e. 12 coplanar lines passing through 9 Eckardt points

In [11]:
# Build all the lines through three Eckardt points that are not contained is SE18
lines_three_eck = []
all_planes = []
for i in range(16):
    for j in range(i+1, 17):
        for k in range(j+1, 18):
            point1 = SE18.eckardt_points[i]
            point2 = SE18.eckardt_points[j]
            point3 = SE18.eckardt_points[k]   
            line12_pl = Point(matrix([point1, point2]).minors(2))
            line23_pl = Point(matrix([point2, point3]).minors(2))
            if line12_pl == line23_pl and line12_pl not in [line.plucker for line in lines_three_eck]:
                planes = get_two_planes_containing_line([point1, point2])
                for plane in planes:
                    if plane not in all_planes:
                        all_planes.append(plane)
                lines_three_eck.append(Line(planes, [point1, point2], line12_pl))
len(lines_three_eck)

42

In [12]:
# Find all planes containing the lines through three Eckardt points
planes = []
for line in lines_three_eck:
    for line2 in line.get_all_lines_incident_to_self(lines_three_eck):
        planes.append(line.get_plane_containing_another_incident_line(line2))
planes = list(set(planes))
len(planes)

31

In [13]:
# The four planes we are looking for are those containing exactly 12 lines on them
c9_planes = [plane for plane in planes if [line.is_on_plane(plane) for line in lines_three_eck].count(True) == 12]
c9_planes

[x + (-i_rad3 - 1)*y - t,
 x + (-1/4*i_rad3 - 1/4)*y + (1/4*i_rad3 + 1/4)*z,
 x + (i_rad3 - 1)*y - t,
 x + (1/4*i_rad3 - 1/4)*y + (-1/4*i_rad3 + 1/4)*z]

In [14]:
nor_subs = Eck_perms_group.normal_subgroups()

In [15]:
%time all_subgroups = Eck_perms_group.subgroups()

CPU times: user 1min 24s, sys: 138 ms, total: 1min 24s
Wall time: 1min 24s


In [16]:
# all elements of H have order 3
H = [el for el in nor_subs if el.order()==27][0]
H_gens_indices = [Eck_perms.index(gen) for gen in H.gens_small()]
H_gens = H.gens_small()
H.structure_description(), list(set([el.order() for el in H if el.cycle_tuples() != []])) == [3]

('C3 x C3 x C3', True)

In [17]:
S4 = Eck_perms_group.quotient(H)
S4.is_isomorphic(SymmetricGroup(4))

True

In [18]:
# We now want explicitly a subgroup isomorphic to S4

In [19]:
subS4 = [el for el in all_subgroups if el.is_isomorphic(S4)][0]
f = SymmetricGroup(4).isomorphism_to(subS4)
S4_gens = [f(gen) for gen in SymmetricGroup(4).gens_small()]
S4_gens_indices = [Eck_perms.index(gen) for gen in S4_gens]

In [20]:
gens = H_gens+S4_gens
gens_indices = H_gens_indices+S4_gens_indices
simms = [simm_SE18[index][0] for index in gens_indices]

In [21]:
# H fixes the planes, S4 acts as S4 on the planes
perms = []
for simm in simms:
    coord_change = change_coordinates(simm)
    new_planes = [plane.subs(coord_change) for plane in c9_planes]
    perm_gen = []
    for new_plane in new_planes:
        for old_plane in c9_planes:
            if matrix([plane_coefficients(old_plane), plane_coefficients(new_plane)]).minors(2) == [0,0,0,0,0,0]:
                perm_gen.append(c9_planes.index(old_plane)+1)
    perms.append(Permutation(perm_gen).to_permutation_group_element())   
perms

[(), (), (), (1,3,2,4), (1,3)]

In [22]:
gens

[(2,3,15)(6,7,13)(10,11,17),
 (1,4,16)(2,3,15)(9,18,12)(10,11,17),
 (5,8,14)(6,13,7)(9,18,12)(10,17,11),
 (1,9,2,17)(3,11,4,12)(5,7)(6,14,13,8)(10,16,18,15),
 (3,15)(5,17)(6,18)(7,9)(8,11)(10,14)(12,13)]

In [23]:
# Study how the simmetries permute the 27 lines

In [24]:
%%time
lines_perms = [Permutation(SE18.apply_proj_to_lines(simm[0])).to_permutation_group_element() for simm in simm_SE18]
lines_perms_group = PermutationGroup(lines_perms)
lines_perms_group.order(), lines_perms_group.structure_description()

(648, '(((C3 x C3 x C3) : (C2 x C2)) : C3) : C2')

In [25]:
gens_lines = [lines_perms[index] for index in gens_indices]
[get_permuted_extended_L_set(perm) for perm in gens_lines], gens_lines

([('F13', 'F24', 'G4', 'E1', 'F35', 'F15'),
  ('F56', 'F13', 'E1', 'F14', 'E3', 'F26'),
  ('E1', 'G4', 'E2', 'G3', 'F45', 'E6'),
  ('F15', 'G1', 'E6', 'F26', 'E4', 'F13'),
  ('E1', 'G4', 'F24', 'F13', 'F34', 'F45')],
 [(1,14,9)(2,10,19)(3,23,11)(4,12,26)(5,16,7)(6,8,21)(13,25,24)(15,27,18)(17,20,22),
  (1,27,2)(5,21,17)(6,20,16)(7,8,22)(9,15,19)(10,14,18),
  (3,25,26)(4,23,24)(5,6,22)(7,21,20)(8,17,16)(11,13,12),
  (1,16)(2,6)(3,4,23,25)(5,14,22,15)(7,9,21,10)(8,19,17,18)(12,26,13,24)(20,27),
  (2,19)(3,22)(4,7)(5,25)(6,26)(8,13)(9,14)(11,16)(12,17)(18,27)(20,24)(21,23)])